<a href="https://colab.research.google.com/github/teadrinke/LangChain/blob/main/Chatbot_using_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-core langgraph>0.2.27

In [ ]:
!pip install -qU langchain-cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.2/411.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.4 MB/s eta 0:00:00


In [ ]:
import langgraph

In [ ]:
!pip langgraph --version

ERROR: unknown command "langgraph"


In [ ]:
print(langgraph.__version__)

AttributeError: module 'langgraph' has no attribute '__version__'

In [ ]:
import getpass
import os

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere")

from langchain_cohere import ChatCohere

model = ChatCohere(model = "command")

Enter API key for Cohere··········


In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content = "Hi, I'm Bob")])

AIMessage(content="Hi Bob! I'm Coral, very nice to meet you. If you have any questions, let me know and I'll do my best to help you out.", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e4ef7ebf-1d24-4590-adcc-a691cbaabfff', 'token_count': {'input_tokens': 67.0, 'output_tokens': 34.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': 'e4ef7ebf-1d24-4590-adcc-a691cbaabfff', 'token_count': {'input_tokens': 67.0, 'output_tokens': 34.0}}, id='run-8f163de9-7528-4743-8469-548837779bcd-0', usage_metadata={'input_tokens': 67, 'output_tokens': 34, 'total_tokens': 101})

In [ ]:
model.invoke([HumanMessage(content = "What's my name?")])

AIMessage(content="I don't know your name as we haven't been introduced yet. I'd love to know yours, so we can start on a more formal note :) \n\nCan you share your name with me? No worries if you'd prefer to stay anonymous, feel free to let me know if I can assist you with anything else instead!", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '136ea558-78c5-4510-9554-59fa88aaac0c', 'token_count': {'input_tokens': 67.0, 'output_tokens': 68.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '136ea558-78c5-4510-9554-59fa88aaac0c', 'token_count': {'input_tokens': 67.0, 'output_tokens': 68.0}}, id='run-8d1df649-e711-4924-8cbc-add758628fdc-0', usage_metadata={'input_tokens': 67, 'output_tokens': 68, 'total_tokens': 135})

To maintain the context you have to pass the entire convo history to the model

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content = "Hi, I am Bob"),
        AIMessage(content = "Hello Bob, How can I assist you today?"),
        HumanMessage(content = "What is my name?")
    ]
)

AIMessage(content="Your name is Bob. Isn't that right?", additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5ecb733c-9bb0-4ef7-b945-7ec4d07efdd9', 'token_count': {'input_tokens': 89.0, 'output_tokens': 11.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '5ecb733c-9bb0-4ef7-b945-7ec4d07efdd9', 'token_count': {'input_tokens': 89.0, 'output_tokens': 11.0}}, id='run-cd04d1ea-f477-4047-989e-265fba8b24c0-0', usage_metadata={'input_tokens': 89, 'output_tokens': 11, 'total_tokens': 100})

In [ ]:
!pip install MessageState

ERROR: Could not find a version that satisfies the requirement MessageState (from versions: none)
ERROR: No matching distribution found for MessageState


In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
# from langgraph.graph.new_location import MessageState

workflow = StateGraph(state_schema = MessagesState)

def call_model(state: MessagesState):
  response = model.invoke(state["messages"])
  return {"messages" : response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [ ]:
config = {"configurable" : {"thread_id":"abc123"}}

In [ ]:
query = "Hi, I am Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages" : input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Nice to meet you, Bob! I'm Coral, a brilliant, sophisticated, AI-assistant chatbot trained to assist human users by providing thorough responses. If you have any questions, please let me know, and I'll do my best to help you out!


In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages" : input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob! But, would you like to know some fun facts about the name Bob? 

Or, would you like me to answer a different question entirely? Let's discuss further about your preferences and I can attempt to help with any information you may require!


In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to best of your abilities."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema = MessagesState)

def call_model(state: MessagesState):
  prompt = prompt_template.invoke(state)
  response = model.invoke(prompt)
  return {"messages" : response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer = memory)

In [ ]:
config = {"configurable" : {"thread_id" : "abc345"}}
query = "Hi, I am Jim."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages" : input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Arrr, Jim! I be Coral, yer pirate AI-assistant chatbot, and I be trained to answer yer questions and help in any way I can. What be yer first task for ye olde Coral?


In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Ah, Jim, yer a pirate through and through, always wantin' to know things. 

And, yessir, yer name be Jim! Or, to translate to pirate-speak, it be Jim the Swabby. 

What else can I do for you, Jim?


In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant. Answer all questions to the best of your ability in {language}"

        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
  messages : Annotated[Sequence[BaseMessage], add_messages]

  language: str

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable" : {"thread_id" : "abc456"}}
query = "Hi, I'm Bob"
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola Bob! Me llamo Coral, y me encanta ayudarte. Que tengas un excelente día. 

If you have any questions, feel free to ask!


Note that the entire state is persisted, so we can omit parameters like language if no changes are desired

In [ ]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You introduced yourself as "Bob".

¿Qué tal es "Bob" como se escribe en español?: "Bób"? 

Can I help you with anything else?


We cannot send full list of message history to the chatbot hence we have to trim or limit our messages history size

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens = 65,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
  trimmed_messages = trimmer.invoke(state["messages"])
  prompt = prompt_template.invoke(
      {"messages" : trimmed_messages, "language" : state["language"]}
  )
  response = model.invoke(prompt)
  return {"messages" : [response]}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob. Would you like me to ask you questions about Bob, or would you like to talk about something else?


In [ ]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked for the answer to 2 + 2, which I answered as 4.


In [ ]:
config = {"configurable" : {"thread_id" : "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

Sure|,| Todd|!| Here|'s| a| joke| for| you|:|

Why| was| the| computer| cold|?|

Because| it| left| its| Windows| open|!|

Hope| you| found| that| joke| to| be| as| cool| as| a| computer|!| If| you|'d| like|,| I| can| tell| you| another| one|.||